<a href="https://colab.research.google.com/github/koffer9/NLP/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
x=int(input('input is '))

def stepcount(x):
	r=int(pow(x-1,0.5)/2+0.5)
	c=x-pow(2*r-1,2)
	n=abs(c%(2*r)-r)+r
	return n

print('steps needed back to square 1 is ',stepcount(x))

input is 30
steps needed back to square 1 is  5


In [2]:
# encoding=utf-8
import jieba


example='你们今天/明天/后天会不会到大学/中学/小学里去打球/滑冰/骑车呀？'
print('请输入一段包含不同选项的句子，各选项直接用分隔符/分开')
print('例句：',example)
print('或者直接回车：')
text=input()
if text=="":
    text=example
print('输入是:')
print(text) #把输入返回一次
seg_list = jieba.lcut(text)  # 默认是精确模式
l=len(seg_list) #拆词列表长度
n=0
ex_location=[]
ex_options=[]
seg_stat=[]

#识别拆词列表各部分的属性，判断出可选项起始位置和选项数量
for i in range(0,l):
    if ((i==0 and seg_list[1]=='/') or (i>0 and i<l-1 and seg_list[i-1]!='/' and seg_list[i+1]=='/')) and seg_list[i]!='/':
        ex_location.append(i)
        n+=1
        seg_stat.append('op_head') #可选项第一选项
    elif (i>0 and i<l-1 and seg_list[i-1]=='/' and seg_list[i+1]=='/') and seg_list[i]!='/':
        n+=1
        seg_stat.append('op_mid') #可选项中间选项
    elif ((i==l-1 and seg_list[i-1]=='/') or (i>0 and i<l-1 and seg_list[i-1]=='/' and seg_list[i+1]!='/')) and seg_list[i]!='/':
        n+=1
        ex_options.append(n)
        n=0
        seg_stat.append('op_tail') #可选项末尾选项
    elif ((i==0 and seg_list[1]!='/') or (i==l-1 and seg_list[i-1]!='/') or (i>0 and i<l-1 and seg_list[i-1]!='/' and seg_list[i+1]!='/')) and seg_list[i]!='/':
        n=0
        seg_stat.append('fixed') #固定部分
    else:
        seg_stat.append('separation') #分隔符
#print('found ex_location: ',ex_location) #识别可选项的起始位置
#print('found ex_options: ',ex_options) 识别可选项的选项数量
#print(seg_stat)拆解后各词组性质

#识别输出数量
total=1
for ex_option in ex_options:
    total=total*ex_option

#选择性输出部分
print('输出是：')
for i in range(0,total):
    ex_accumulate=[]
    ex_choice=[]
    for j in range(0,len(ex_options)): #计算出每个选项的累积乘数，根据累计乘数取余算出所有选项组合
        ex_choice.append(0)
        if j==0:
            ex_accumulate.append(ex_options[j])
            ex_choice[j]=i%ex_options[j]
        else:
            ex_accumulate.append(ex_accumulate[j-1]*ex_options[j])
            ex_choice[j]=int(i/ex_accumulate[j-1])%ex_options[j]
    #print(i,' ',ex_choice[0],' ',ex_choice[1],' ',ex_choice[2])
    sentence=[]
    for m in range(0,l): #从最初的拆词列表内选择输出需要的词组，组合输出
        if seg_stat[m]=='fixed':
            sentence.append(seg_list[m])
        else:
            for n in range(0,len(ex_options)):
                if m==ex_location[n]:
                    sentence.append(seg_list[ex_location[n]+2*ex_choice[n]])
    print(i+1,'：',"".join(sentence))





ModuleNotFoundError: ignored

In [10]:
simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article => 一个 | 这个
noun => 女人 |  篮球 | 桌子 | 小猫
verb => 看着 |  坐在 | 听着 | 看见 
Adj => 蓝色的 |  好看的 | 小小的
"""
#define list needed
header=['']*len(simple_grammar.splitlines())
value=['']*len(simple_grammar.splitlines())
words=[['']*1 for x in range(len(simple_grammar.splitlines()))]
result=[]

def split():  #to split the grammar
	i=0
	for line in simple_grammar.splitlines():
		if line.count('=>')>0:
			header[i]=line.split('=>')[0].strip()
			words[i]=line.split('=>')[1].split('|')
		for j in range(len(words[i])):
			words[i][j]=words[i][j].lstrip()
			words[i][j]=words[i][j].rstrip()
		#print(header[i],words[i])
		i+=1
	#print('---splitted---')

def rand(): #to randomize the options
	import random
	for i in range(len(simple_grammar.splitlines())):
		value[i]=words[i][random.randint(0,len(words[i])-1)].split(' ')
		#print(header[i],'=',value[i])
	#print('---randomized---')

def breakdown(result): #to breakdown the components
	for m in range(len(result)):
		for n in range(len(simple_grammar.splitlines())):
			if result[m]==header[n]:
				del result[m]
				for v in range(len(value[n])):
					result.insert(m+v,value[n][v])
				m=m+v
	#print(result)
	#print('---brokendown---')
	return result

def generator(input):
	print('Grammar Input is:')
	print(simple_grammar)
	split()
	rand()
	for i in range(len(simple_grammar.splitlines())):
		if input==header[i]:
			result=value[i]	
	exhaust=False
	cycle=0
	while exhaust!=True: #this is a repeat to exhaust components
		rand()
		breakdown(result)
		#print(cycle,' ',result)
		exhaust=True
		for m in range(len(result)):
			for n in range(len(simple_grammar.splitlines())):
				if result[m]==header[n]:
					#print(result[m],' is not exhausted')
					exhaust=False
					cycle+=1
					break
	i=0
	tail=False
	while tail!=True: #to remove null and repeated adj
		if i>0 and (result[i]==result[i-1] or result[i]=='null'):
			del result[i]
			i+=-1
		tail=i==len(result)-1
		i+=1
			
	print('Random Output is:')
	print(''.join(result))

generator('sentence')


Grammar Input is:

sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article => 一个 | 这个
noun => 女人 |  篮球 | 桌子 | 小猫
verb => 看着 |  坐在 | 听着 | 看见 
Adj => 蓝色的 |  好看的 | 小小的

Random Output is:
这个桌子看见这个蓝色的小猫
